In [1]:
import sys
import os
import math
from pyspark.rdd import RDD
from pyspark import SparkConf, SparkContext
os.environ['JAVA_HOME'] = 'C:\Program Files\Java\jdk1.8.0_301'
os.environ['PYSPARK_PYTHON'] = sys.executable
os.environ['PYSPARK_DRIVER_PYTHON'] = sys.executable

In [2]:
sc.stop()
conf = SparkConf().setMaster("local").setAppName("LSH")
sc = SparkContext(conf=conf)

In [3]:
all_file = sc.wholeTextFiles(os.path.join('data', 'athletics'))

In [6]:
all_file.collect()[0]

('file:/C:/Users/Kyle/MDA/term_project/data/athletics/001.txt',
 'Claxton hunting first major medal\n\nBritish hurdler Sarah Claxton is confident she can win her first major medal at next month\'s European Indoor Championships in Madrid.\n\nThe 25-year-old has already smashed the British record over 60m hurdles twice this season, setting a new mark of 7.96 seconds to win the AAAs title. "I am quite confident," said Claxton. "But I take each race as it comes. "As long as I keep up my training but not do too much I think there is a chance of a medal." Claxton has won the national 60m hurdles title for the past three years but has struggled to translate her domestic success to the international stage. Now, the Scotland-born athlete owns the equal fifth-fastest time in the world this year. And at last week\'s Birmingham Grand Prix, Claxton left European medal favourite Russian Irina Shevchenko trailing in sixth spot.\n\nFor the first time, Claxton has only been preparing for a campaign ove

In [9]:
def trigram_mapper(lines):
    # lines[0]: file path
    line = lines[1] # text
    line = line.replace("\n","")
    line = line.replace(",","")
    line = line.replace(".","")
    line = line.replace(":","")
    line = line.replace("-","")
    line = line.replace("\\","")
    line = line.replace("\"","")
    line = line.replace("\'","")
    wordlist = list()
    for word in line.split(" "):
        wordlist.append(word)
    trigram = list()
    for i in range(len(wordlist)):
        if(i+2<len(wordlist)):
            trigram.append(((wordlist[i], wordlist[i+1], wordlist[i+2]),0))
    return trigram

In [10]:
trigram = all_file.map(trigram_mapper).collect()